Здравствуйте! Сейчас сделаю небольшое введение о том, какие подходы бывают к решению данной задачи, а также о способе который я выбрал.

Для создания рекомендационных систем используются алгоритмы Collaborative Filtering, которые разделяются на 2 большие группы: User-Based и Item-Based.

В первой: алгоритм находит похожих на тестового пользователей, основываясь на их отзывах. Похожесть определяется тем, что тестовый пользователь оценивает одинаково с похожими пользователями большинство вещей (не уточняли в задаче, что именно считаем за items, поэтому называю вещами). Затем, на основании того, что похожие пользователи высоко оценили, для тестового пользователя делается рекомендация (важно проверить только, что он еще не использвал эту вещь сам, то есть не ставил ей рейтинг).

Второй способ: искать не похожих пользователей, а похожие вещи; например, если сразу много людей одинаково хорошо оценили 2 вещи, то они будут считаться похожими. Далее для тестового пользователя смотрят на вещи, которые он высоко оценил, находят для них похожие и рекомендуют.

Я избрал для решения задачи первый путь, потому что для него можно найти больше информации в интернете (а сам я столкнулся с подобной задачей впервые). По этой же причине, что опыта совсем не было, прочел несколько статей, где как раз и наткнулся на пример решения похожей задачи. Я разобрался, как у них все работает и применил их решение в нашей ситуации.

Ссылка на ресурсы, которыми пользовался:

Основная идея: https://towardsdatascience.com/build-a-user-based-collaborative-filtering-recommendation-engine-for-anime-92d35921f304

https://realpython.com/build-recommendation-engine-collaborative-filtering/

Небольшой обзор: https://www.coursera.org/lecture/machine-learning-with-python/collaborative-filtering-4y9I1

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
data = pd.read_csv('/home/cats/Files/Homework/НИР/JB-2021-Autumn-Internship-Task-master/data/dataset.csv')
data = data.sort_values(['timestamp'])

In [3]:
train = data[:80000]
test = data[80000:]

In [4]:
train.head()

,user_id,item_id,rating,timestamp
217,259,255,4,874724710
83968,259,286,4,874724727
43030,259,298,4,874724754
21399,259,185,4,874724781
82658,259,173,4,874724843


In [5]:
test.head()

,user_id,item_id,rating,timestamp
1346,3,245,1,889237247
27978,3,355,3,889237247
1260,3,335,1,889237269
38673,3,322,3,889237269
3761,3,323,2,889237269


Для решения потребуются дополнительные библиотеки. Одна из них для нахождения "похожести", а вторая для сортировки:

In [6]:
# for cosine_similarity
from sklearn.metrics.pairwise import cosine_similarity

# For sorting using key=operator.itemgetter
import operator

Далее заведем функцию, которая определяет по пользователю рекомендации для него. Она принимает id пользователя и число рекомендаций, которое надо вывести.

In [81]:
def recommend(user_id, k):
    # Создадим матрицу со строками user_id и столбцами item_id. Это тоже объект DataFrame.
    # Она поможет в дальнейшем понимать, кто из пользователей похож.
    rating_matrix = train.pivot_table(index='user_id', columns='item_id', values='rating')
    # Заполним нулями элементы, которые не заданы (NaN). Таких элементов в матрице много, т.к.
    # каждый пользователь выставлял рейтинг только ограниченному малому числу вещей.
    rating_matrix = rating_matrix.fillna(0)
    
    # Выведем, чтобы посмотреть
    # print(rating_matrix.head())
    
    # Строка пользователя, для которого делаем рекомендации (назовем его тестовым). 
    # Здесь отображено, каким вещам он уже постаивл рейтинг и каким нет
    user = rating_matrix[rating_matrix.index == user_id]
    
    # Холодный старт - нет информации о тестовом пользователе. Выведем наиболее часто встречающиеся.
    if user.shape[0] == 0:
        return train['item_id'].value_counts().index[:k].tolist()

    # Строки остальных пользователей
    other_users = rating_matrix[rating_matrix.index != user_id]

    # Используя библиотеку, находим сходство между данным пользователем и остальными с помощью cosine similarity.
    similarities = cosine_similarity(user, other_users)[0].tolist()

    # Список всех пользователей, кроме данного в виде списка
    indices = other_users.index.tolist()

    # Создаем пары: пользователь и число сходства с тестовым пользователем.
    index_similarity = dict(zip(indices, similarities))

    # Сортируем по сходству. Пользователи распологаются по убыванию сходства с тестовым.
    index_similarity_sorted = sorted(index_similarity.items(), key=operator.itemgetter(1))
    index_similarity_sorted.reverse()
    
    # Берем первых 3-х пользователей (они имеют наибольшее сходство с данным в задаче).
    # Число выбираемых похожих пользователей можно менять
    top_similar_users_similarities = index_similarity_sorted[:3]
    similar_users_indices = [u[0] for u in top_similar_users_similarities]
    
    # На данный момент имеем список с id пользователей, которые наиболее похожи. Далее идея в следующем:
    # 1) Вычислить средний рейтинг среди похожих пользователей, на вещи, которые они использовали.
    # 2) Найти какие вещи не использовал (нет рейтинга) тестовый пользователь и посмотреть для них средний рейтинг
    # похожих пользователей. Вещи с максимальным рейтингом порекомендовать.
    
    
    # Получаем строки матрицы рейтингов для одинаковых пользователей
    similar_users = rating_matrix[rating_matrix.index.isin(similar_users_indices)]
    # Посчитаем рейтинг, который похожие пользователи дали для вещей в среднем
    similar_users = similar_users.mean(axis=0)
    # Конвертируем в элемент dataframe для удобства
    similar_users_df = pd.DataFrame(similar_users, columns=['mean'])
    
    
    # Транспонируем строку пользователя, т.к. это удобнее для дальнейшей работы (у похожих пользователей рейтинги
    # для вещей записаны вертикально)
    user_transposed = user.transpose()
    # Для красоты сменим название с имени тестового пользователя на 'rating'
    user_transposed.columns = ['rating']
    # Удаляем у тестового пользователя в столбце все значения ненулевого рейтинга
    # (оставляем те, которыми не пользовался)
    user_transposed = user_transposed[user_transposed['rating'] == 0]
    # Конвертируем таблицу из нулевых рейтингов и вещей в список вещей.
    # Там все, чем тестовый пользователь не пользовался
    unused_items = user_transposed.index.tolist()
    
    # В таблице рейтингов похожих пользователей отбираем строки только с теми вещами,
    # которыми тестовый не пользовался
    similar_users_df_filtered = similar_users_df[similar_users_df.index.isin(unused_items)]
    # Сортируем таблицу по рейтингу (сверху наибольшее значение для среднего рейтинга похожих пользователей)
    similar_users_df_ordered = similar_users_df.sort_values(by=['mean'], ascending=False)
    # Берем первые 5 строк таблицы. Индексы - названия вещей.
    # Эти вещи имеют наибольший рейтинг у похожих пользователей и их еще не использовал тестовый пользователь.
    top_n_items = similar_users_df_ordered.index[:k].tolist()
    

    return top_n_items

Заведем две функции. Первая будет считать, как много рекомендаций попало в массив вещей, которыми воспользовался тестовый пользователь. Вторая будет выводить число - отношение числа рекомендаций, которые получили правильно, к идеальному числу рекомендаций.

Здесь, однако, есть небольшая загвоздка: функции проверки никак не учитывают то, что тестовый пользователь мог воспользоваться вещью, но поставить ей низкий рейтинг. Хорошо бы учитывать с большим весом угадывание рекомендации, которой тестовый пользователь поставил больший рейтинг.

In [82]:
def average_precision(actual, recommended, k=30):
    ap_sum = 0
    hits = 0
    for i in range(k):
        product_id = recommended[i] if i < len(recommended) else None
        if product_id is not None and product_id in actual:
            hits += 1
            ap_sum += hits / (i + 1)
    return ap_sum / k


def normalized_average_precision(actual, recommended, k=30):
    actual = set(actual)
    if len(actual) == 0:
        return 0.0

    ap = average_precision(actual, recommended, k=k)
    ap_ideal = average_precision(actual, list(actual)[:k], k=k)
    return ap / ap_ideal

In [84]:
scores = []
for user in tqdm(test['user_id'].unique()):
    # Для каждого пользователя из тестовой выборки смотрим набор вещей, для которых выставлен рейтинг
    actual = list(test[test['user_id'] == user]['item_id'])
    recommended = recommend(user, 50)
    
    scores.append(normalized_average_precision(actual, recommended, 50))

np.mean(scores)

100%|█████████████████████████████████████████| 301/301 [00:30<00:00,  9.73it/s]


0.13205455894655577